In [12]:
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

## Importing the data

In [11]:
# Data location
data_dir = Path('./input')

# training
train_dir = data_dir / 'train'

train_pneumonia = [['input/train/PNEUMONIA/' + i, 1]  for i in os.listdir(train_dir / 'PNEUMONIA') if i.endswith(".jpeg")]
train_healthy   = [['input/train/NORMAL/' + i, 0]     for i in os.listdir(train_dir / 'NORMAL') if i.endswith(".jpeg")]
train_pneumonia.extend(train_healthy)

df_train = pd.DataFrame(train_pneumonia, columns = ['img', 'label'], index=None)
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

# validation
val_dir = data_dir / 'val'

val_pneumonia = [['input/val/PNEUMONIA/' + i, 1]  for i in os.listdir(val_dir / 'PNEUMONIA') if i.endswith(".jpeg")]
val_healthy   = [['input/val/NORMAL/' + i, 0]     for i in os.listdir(val_dir / 'NORMAL') if i.endswith(".jpeg")]
val_pneumonia.extend(val_healthy)

df_val = pd.DataFrame(val_pneumonia, columns = ['img', 'label'], index=None)
df_val = df_val.sample(frac=1, random_state=42).reset_index(drop=True)

# test
test_dir = data_dir / 'test'

test_pneumonia = [['input/test/PNEUMONIA/' + i, 1]  for i in os.listdir(test_dir / 'PNEUMONIA') if i.endswith(".jpeg")]
test_healthy   = [['input/test/NORMAL/' + i, 0]     for i in os.listdir(test_dir / 'NORMAL') if i.endswith(".jpeg")]
test_pneumonia.extend(test_healthy)

df_test = pd.DataFrame(test_pneumonia, columns = ['img', 'label'], index=None)
df_test = df_test.sample(frac=1, random_state=42).reset_index(drop=True)
print(df_test.shape, df_train.shape, df_val.shape)
df_train.head()

(624, 2) (5216, 2) (16, 2)


,img,label
0,input/train/PNEUMONIA/person299_bacteria_1418....,1
1,input/train/PNEUMONIA/person1415_virus_2425.jpeg,1
2,input/train/NORMAL/NORMAL2-IM-0416-0001-0002.jpeg,0
3,input/train/NORMAL/NORMAL2-IM-0757-0001.jpeg,0
4,input/train/PNEUMONIA/person1924_bacteria_4833...,1


In [19]:
train_counts = df_train['label'].value_counts()
val_counts = df_val['label'].value_counts()
test_counts = df_test['label'].value_counts()
print(train_counts, val_counts, test_counts)

label
1    3875
0    1341
Name: count, dtype: int64 label
1    8
0    8
Name: count, dtype: int64 label
1    390
0    234
Name: count, dtype: int64
